## Mathematical Problem Formulation

The Poisson Equation is a boundary problem described by the following equation:

\begin{align}
-\Delta u(\boldsymbol{x}) &= f(\boldsymbol{x}),\quad \ \ \boldsymbol{x}\in \Omega \\
u(\boldsymbol{x}) &= u_D(\boldsymbol{x}),\quad  \boldsymbol{x}\in \partial \Omega\ .
\end{align}

Here $u(\boldsymbol{x})$ is the unknown function, $f(\boldsymbol{x})$ is a prescribed function, $\Omega$ is the spatial domain of the problem, and $\partial \Omega$ is the boundary of $\Omega$.

In two dimensions with the coordinate $x$ and $y$ this problem can be written as:
\begin{align}
\dfrac{\partial^2 u}{\partial x^2} + \dfrac{\partial^2 u}{\partial y^2} &= -f(x,y)\ .
\end{align}

The unknown function $u$ is now a function of two variables $u=u(x,y)$, defined over a two-dimensional domain $\Omega$.

Solving a boundary-value problem such as the Poisson equation in FEniCS consists of the following steps:
1. Identify the computational domain ($\Omega$), the PDE, its boundary conditions, and source terms ($f$).
2. Reformulate the PDE as a finite element variational problem.
3. Write a Python program which defines the computational domain, the variational problem, the boundary conditions, and source terms, using the corresponding FEniCS abstractions.
4. Call FEniCS to solve the boundary-value problem and, optionally, extend the program to compute derived quantities such as fluxes and averages, and visualize the results.

### Finite element variational formulation

FEniCS is based on the finite element method, which is a general and efficient mathematical machinery for the numerical solution of PDEs. The starting point for the finite element methods is a PDE expressed in *variational form*.

The basic recipe for turning a PDE into a variational problem is to multiply the PDE by a function $v$, integrate the resulting equation over the domain $\Omega$, and perform integration by parts of terms with second-order derivatives. The function $v$ which multiplies the PDE is called a *test function*. The unknown function $u$ to be approximated is referred to as a *trial function*. The terms trial and test functions are used in FEniCS programs too. The trial and test functions belong to certain so-called *function spaces* that specify the properties of the functions.

In the present case, we first multiply the Poisson equation by the test function $v$ and integrate over $\Omega$:
\begin{align}
-\int_{\Omega}(\Delta u)v\ dx &= \int_{\partial\Omega} f v\ dx
\end{align}
We here let $dx$ denote the differential element for integration over the domain $\Omega$. We will later let $ds$ denote the differential element for integration over the boundary of $\Omega$.

A common rule when we derive variational formulations is that we try to keep the order of the derivatives of $u$ and $v$ as small as possible. Here, we have a second-order spatial derivative of $u$, which can be transformed to a first-derivative of $u$ and $v$ by applying the technique of [integration by parts](https://en.wikipedia.org/wiki/Integration_by_parts). Applying this to the right-hand side of the integration above yields:
\begin{align}
-\int_{\Omega}(\Delta u)v\ dx &= \int_{\Omega}\nabla u \cdot \nabla v\ dx - \int_{\partial\Omega}\dfrac{\partial u}{\partial n}v\ ds
\end{align}
Here $\dfrac{\partial u}{\partial n}=\nabla u \cdot n$ is the derivative of $u$ in the outward normal direction $n$ on the boundary.
Another feature of variational formulations is that the test function $v$ is required to vanish on the parts of the boundary where the solution $u$ is known  [this book](./fem-book-4screen.pdf) explains in detail why this requirement is necessary). In the present problem, this means that $v=0$ on the whole boundary $\partial \Omega$.

This means that in the last equation the second term on the left-hand side vanishes, which results in:
\begin{align}
-\int_{\Omega}(\Delta u)v\ dx &= \int_{\Omega}\nabla u \cdot \nabla v\ dx
\end{align}

If we require that this equation holds for all test functions $v$ in some suitable space $\hat{V}$, the so-called *test space*, we obtain a well-defined mathematical problem that uniquely determines the solution $u$ which lies in some (possibly different) function space $V$, the so-called *trial space*. We refer to the equation above as the *weak form* or *variational form* of the original boundary-value problem.

The proper statement of our variational problem now goes as follows: find $u\in V$ such that
\begin{align}
\int_{\Omega} \nabla u\cdot \nabla v\ dx &= \int_{\Omega} f v\ dx \quad \forall v\in \hat{V}\ .
\end{align}
The trial and test spaces $V$ and $\hat{V}$ are in the present problem defined as:
\begin{align}
V &= \lbrace v\in H^1(\Omega) : v=u_D\ \text{on}\ \partial \Omega \rbrace\ , \\
\hat{V} &= \lbrace v\in H^1(\Omega) : v=0\ \text{on}\ \partial \Omega \rbrace\ .
\end{align}

In short, $H^1(\Omega)$ is the mathematically well-known Sobolev space containing functions $v$ such that $v^2$ and $|\nabla v|^2$ have finite integrals over $\Omega$ (essentially meaning that the functions are continuous). The solution of the underlying PDE must lie in a function space where the derivatives are also continuous, but the Sobolev space $H^1(\Omega)$ allows functions with discontinuous derivatives. This weaker continuity requirement of $u$ in the variational statement, as a result of the integration by parts, has great practical consequences when it comes to constructing finite element function spaces. In particular, it allows the use of piecewise polynomial function spaces; i.e., function spaces constructed by stitching together polynomial functions on simple domains such as intervals, triangles, or tetrahedrons.

The variational problem is a continuous problem: it defines the solution $u$ in the infinite-dimensional function space $V$. The finite element method for the Poisson equation finds an approximate solution of the variational problem by replacing the infinite-dimensional function spaces $V$ and $\hat{V}$ by discrete (finite-dimensional) trial and test spaces $V_h\in V$ and $\hat{V_h}\in\hat{V}$. The discrete variational problem reads: find $u_h\in V_h\in V$ such that
\begin{align}
\int_{\Omega}\nabla u_h\cdot \nabla v\ dx &= \int_{\Omega}f v\quad \forall v\in \hat{V_h}\in \hat{V}\ .
\end{align}

This variational problem, together with a suitable definition of the function spaces $V_h$ and $\hat{V_h}$, uniquely define our approximate numerical solution of Poisson’s equation. Note that the boundary conditions are encoded as part of the trial and test spaces. The mathematical framework may seem complicated at first glance, but the good news is that the finite element variational problem looks the same as the continuous variational problem, and FEniCS can automatically solve variational problems like!

In the following parts, we will drop the subscript $_h$ from $u_h$ and $V_h$ and instead call $u$ the solution for the discrete problem and $V$ as the discrete finite element function space in which we seek our solution. Whenever we will look at the exact solution for the continuous problem, we will use $u_e$.

### Abstract finite element variational formulation

It turns out to be convenient to introduce the following canonical notation for variational problems: find $u\in V$ such that
\begin{align}
a(u,\ v) &= L(v)\quad \forall v\in \hat{V}\ .
\end{align}
For the Poisson equation, we have:
\begin{align}
a(u,\ v) &= \int_{\Omega} \nabla u \cdot \nabla v\ dx\ ,\\
L(v) &= \int_{\Omega} f v\ dx\ .
\end{align}

From the mathematics literature, $a(u,v)$ is known as a bilinear form and $L(v)$ as a linear form. We shall, in every linear problem we solve, identify the terms with the unknown $u$ and collect them in $a(u,v)$, and similarly collect all terms with only known functions in $L(v)$. The formulas for $a$ and $L$ can then be expressed directly in our FEniCS programs.

To solve a linear PDE in FEniCS, such as the Poisson equation, a user thus needs to perform only two steps:
1. Choose the finite element spaces $V$ and $\hat{V}$ by specifying the domain (the mesh) and the type of function space (polynomial degree and type).
2. Express the PDE as a (discrete) variational problem: find $u\in V$ such that $a(u,v)=L(v)$ for all $v∈\hat{V}$.

### Choosing a test problem
The Poisson problem has so far featured a general domain $\Omega$ and general functions $u_D$ for the boundary conditions and $f$ for the right-hand side. For our first implementation we will need to make specific choices for $\Omega$, $u_D$, and $f$. It will be wise to construct a problem with a known analytical solution so that we can easily check that the computed solution is correct. Solutions that are lower-order polynomials are primary candidates. Standard finite element function spaces of degree $r$ will exactly reproduce polynomials of degree $r$. And piecewise linear elements ($r=1$) are able to exactly reproduce a quadratic polynomial on a uniformly partitioned mesh. This important result can be used to verify our implementation. We just manufacture some quadratic function in 2D as the exact solution, say
\begin{align}
u_e(x,y) &= 1 + x^2 + 2y^2
\end{align}
By inserting the solution $u_e$ into the Poisson equation, we find that $u_e(x,y)$ is a solution if
\begin{align}
f(x,y)=−6,\quad u_D(x,y)=u_e(x,y)=1+x^2+2y^2,
\end{align}
regardless of the shape of the domain as long as ue is prescribed along the boundary. We choose here, for simplicity, the domain to be the unit square,
\begin{align}
\Omega = [0,1]\times [0,1]\ .
\end{align}
This simple but very powerful method for constructing test problems is called *the method of manufactured solutions*: pick a simple expression for the exact solution, plug it into the equation to obtain the right-hand side (source term $f$), then solve the equation with this right-hand side and using the exact solution as a boundary condition, and try to reproduce the exact solution.

## FEniCS implementation

In [ ]:
from fenics import *
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Create mesh and define function space
mesh = UnitSquareMesh(8, 8)
V = FunctionSpace(mesh, "P", 1)

In [ ]:
# Define boundary condition
u_D = Expression('1 + x[0]*x[0] + 2*x[1]*x[1]', degree=2)
def boundary(x, on_boundary):
    return on_boundary

bc = DirichletBC(V, u_D, boundary)

In [ ]:
# Define variational problem
u = TrialFunction(V)
v = TestFunction(V)
f = Constant(-6.0)
a = dot(grad(u), grad(v))*dx
L = f*v*dx

In [ ]:
# Compute solution
u = Function(V)
solve(a == L, u, bc)

In [ ]:
# Plot solution and mesh
plot(u)
plot(mesh)
plt.show()

In [ ]:
# Save solution to file in VTK format
vtkfile = File('poisson/solution.pvd')
vtkfile << u

In [ ]:
# Compute error in L2 norm
error_L2 = errornorm(u_D, u, 'L2');

In [ ]:
# Compute maximum error at vertices
vertex_values_u_D = u_D.compute_vertex_values(mesh)
vertex_values_u = u.compute_vertex_values(mesh)
error_max = np.max(np.abs(vertex_values_u_D - vertex_values_u))

In [ ]:
# Print errors
print('error_L2  =', error_L2)
print('error_max =', error_max)

## Deflection of a membrane

Our first FEniCS program for the Poisson equation targeted a simple test problem where we could easily verify the implementation. We now turn our attention to a physically more relevant problem with solutions of somewhat more exciting shape.

We want to compute the deflection $D(x,y)$
of a two-dimensional, circular membrane of radius $R$, subject to a load p over the membrane. The appropriate PDE model is

\begin{align}
−T\Delta D&=p\quad \text{with}\ \Omega=\lbrace (x,y)|x^2+y^2\leq R\rbrace\ .
\end{align}

Here, $T$ is the tension in the membrane (constant), and p is the external pressure load. The boundary of the membrane has no deflection, implying $D=0$ as a boundary condition. A localized load can be modeled as a Gaussian function:

\begin{align}
p(x,y)&=\frac{A}{2\pi\sigma}exp\big(−\frac{1}{2}\big(\frac{x−x_0}{\sigma}\big)^2−\frac{1}{2}\big(\frac{y−y_0}{\sigma}\big)^2\big)\ .
\end{align}

The parameter $A$ is the amplitude of the pressure, $(x_0,y_0)$ the localization of the maximum point of the load, and $\sigma$ the “width” of $p$. We will take the center $(x_0,y_0)$ of the pressure to be $(0,R_0)$ for some $0<R_0<R$.

### Scaling the equation

There are many physical parameters in this problem, and we can benefit from grouping them by means of scaling. Let us introduce dimensionless coordinates $\bar{x}=x/R, \bar{y}=y/R$, and a dimensionless deflection $w=D/D_c$, where $D_c$ is a characteristic size of the deflection. Introducing $\bar{R}_0=R_0/R$, we obtain

\begin{align}
−\dfrac{\partial^2 w}{\partial\bar{x}^2}−\dfrac{\partial^2 w}{\partial\bar{y}^2}=\alpha\ exp\big(−\beta^2(\bar{x}^2+(\bar{y}−\bar{R}_0)^2)\big)\quad \text{for}\ \bar{x}^2+\bar{y}^2<1,
\end{align}

with

\begin{align}
\alpha=\frac{R^2A}{2\pi TD_c\sigma},\ \beta=\frac{R^2}{2\sqrt{\sigma}}\ .
\end{align}

With an appropriate scaling, $w$ and its derivatives are of size unity, so the left-hand side of the scaled PDE is about unity in size, while the right-hand side has $\alpha$ as its characteristic size. This suggest choosing $\alpha$ to be unity, or around unity. We shall in this particular case choose $\alpha=4$. (One can also find the analytical solution in scaled coordinates and show that the maximum deflection $D(0,0)$ is $D_c$ if we choose $\alpha=4$ to determine $D_c$.) With $D_c=AR^2/(8\pi\sigma T)$ and dropping the bars we obtain the scaled problem

\begin{align}
−\Delta w&=4\ exp\big(−\beta^2(x^2+(y−R_0)^2)\big),
\end{align}

to be solved over the unit disc with $w=0$ on the boundary. Now there are only two parameters to vary: the dimensionless extent of the pressure, $\beta$, and the localization of the pressure peak, $R_0\in[0,1]$. As $\beta\rightarrow 0$, the solution will approach the special case $w=1−x^2−y^2$.

Given a computed scaled solution $w$, the physical deflection can be computed by
\begin{align}
D=\frac{AR^2}{8\pi\sigma T}w\ .
\end{align}

### Defining the mesh

A mesh over the unit disk can be created by the ```mshr``` tool in FEniCS:

In [ ]:
import dolfin
from mshr import *
domain = Circle(Point(0, 0), 1)
mesh = generate_mesh(domain, 64)
V = FunctionSpace(mesh, "P", 1)

The ```Circle``` shape from ```mshr``` takes the center and radius of the circle as arguments. The second argument to the ```generate_mesh``` function specifies the desired mesh resolution. The cell size will be (approximately) equal to the diameter of the domain divided by the resolution.


### Defining the load
The right-hand side pressure function is represented by an ```Expression``` object. There are two physical parameters in the formula for $f$ that enter the expression string and these parameters must have their values set by keyword arguments:

In [ ]:
beta = 8
R0 = 0.6
p = Expression('4*exp(-pow(beta, 2)*(pow(x[0], 2) + pow(x[1] - R0, 2)))',
               degree=1, beta=beta, R0=R0)

def boundary(x, on_boundary):
    return on_boundary

bc = DirichletBC(V, p, boundary)

The coordinates in Expression objects are always an array x with components ```x[0]```,```x[1]```, and ```x[2]```, corresponding to $x$, $y$, and $z$. Otherwise we are free to introduce names of parameters as long as these are given default values by keyword arguments. All the parameters initialized by keyword arguments can at any time have their values modified. For example, we may set
```python
p.beta = 12
p.R0 = 0.3
```

### Defining the variational problem

The variational problem is the same as in our first Poisson problem, but we may introduce $w$ instead of $u$ as primary unknown and $p$ instead of $f$ as right-hand side function:

In [ ]:
w = TrialFunction(V)
v = TestFunction(V)
a = dot(grad(w), grad(v))*dx
L = p*v*dx
w = Function(V)
solve(a == L, w, bc)

### Plotting the solution

It is of interest to visualize the pressure $p$ along with the deflection $w$ so that we may examine the membrane’s response to the pressure. We must then transform the formula (```Expression```) to a finite element function (```Function```). The most natural approach is to construct a finite element function whose degrees of freedom are calculated from $p$. That is, we interpolate $p$ to the function space $V$:

In [ ]:
p = interpolate(p, V)

Note that the assignment to $p$ destroys the previous ```Expression``` object ```p```, so if it is of interest to still have access to this object, another name must be used for the ```Function``` object returned by interpolate. The two functions ```w``` and ```p``` may be plotted using the built-in ```plot``` command:

In [ ]:
x_array=np.arange(-0.99,1, 0.01)
y_array=np.arange(-0.99,1, 0.01)
i=0
w_array = np.zeros(shape=(len(y_array),len(x_array)))
p_array = np.zeros(shape=(len(y_array),len(x_array)))
w.set_allow_extrapolation(True)
p.set_allow_extrapolation(True)
for j,y in enumerate(y_array):
    for i,x in enumerate(x_array):
        if x**2+y**2<1:
            w_array[j,i]=w((x,y))
            p_array[j,i]=p((x,y))
        else:
            w_array[j,i]=np.nan
            p_array[j,i]=np.nan

In [ ]:
fig,ax=plt.subplots(1,2,figsize=(16,9))
ax[0].set_title(r"Deflection $w$")
cs=ax[0].contourf(x_array,y_array,w_array,500,cmap="jet")
ax[0].contour(x_array,y_array,w_array,10, colors='k')
ax[0].set_aspect(1)
fig.colorbar(cs,ax=ax[0],shrink=0.67)

ax[1].set_title(r"Load $p$")
cs=ax[1].contourf(x_array,y_array,p_array,500,cmap="jet")
ax[1].contour(x_array,y_array,p_array,10, colors='k')
ax[1].set_aspect(1)
fig.colorbar(cs,ax=ax[1],shrink=0.67)

plt.show()

In [ ]:
# Curve plot along x = 0 comparing p and w
tol = 0.001  # avoid hitting points outside the domain
y = np.linspace(-1 + tol, 1 - tol, 101)
points = [(0, y_) for y_ in y]  # 2D points
w_line = np.array([w(point) for point in points])
p_line = np.array([p(point) for point in points])
plt.plot(y, 50*w_line, 'k', linewidth=2)  # magnify w
plt.plot(y, p_line, 'b--', linewidth=2)
plt.grid(True)
plt.xlabel('$y$')
plt.legend(['Deflection ($\\times 50$)', 'Load'], loc='upper left')
plt.show()